# Predict the next seen movies

> The goal of this project is to validate your skills learned during the previous tutorials with the PySpark distributed computing software. You will build and test several implementations of recommendation systems. Here are the parts of the project:
> 
> * Part A: Load and preprocess the dataset. 
> * Part B: Build a first set of recommendation algorithms: naive with recurring pairs, a priori, and fp-growth to infer rules from a dataset.
> * Part C: Implement the PLSI algorithm ("Probabilistic Latent Semantic Indexing"), which is one way to create embeddings from a dataset. These embeddings can be used to fuel a recommendation engine.
> * Part D: Predict the next movies seen by a user in function of the last movies that he/she has seen with the previous implemented algorithms.

# Setup

## Install Spark Environment

> Since you are running on Google Colab, you will need to install Spark by ourselves, every time we run a new session. You need to install Spark, as well as a Java Runtime Environment.  Then you need to setup a few environment variables.

Execute:

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!curl -O https://dlcdn.apache.org/spark/spark-3.2.3/spark-3.2.3-bin-hadoop3.2.tgz
!tar xf spark-3.2.3-bin-hadoop3.2.tgz

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.3-bin-hadoop3.2"

In [ ]:
!pip install -q findspark
import findspark
findspark.init()

or directly with:

In [ ]:
#!pip install pyspark==3.2.3

Create and launch a Spark session with:

In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf

conf = SparkConf().set('spark.ui.port', '4050')

spark = SparkSession.builder.config(conf=conf)\
  .master('local[*]')\
  .getOrCreate()

sc = spark.sparkContext 

In [ ]:
spark

/!\ The Spark UI link is not accessible. Use the optional next session if you want to access it.

Uncomment and execute the following line, if you want to close and stop the created Spark session 

In [ ]:
#spark.stop()

## Optional step: Enable Spark UI access through a secure tunnel

> This step is useful if you want to look at Spark UI.
First, you need to create a free ngrok account : https://dashboard.ngrok.com/login.  
Then connect on the website and copy your AuthToken.

In [ ]:
!uname -a

In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

In [ ]:
!./ngrok authtoken '2HYfdysedjgB5lcoeE8lqqShgIe_sZUm49MquX5okpssVMYQ' # <-------------- change this line !

In [ ]:
get_ipython().system_raw('./ngrok http 4050 &')

In [ ]:
import json
import urllib

ngrok_tunnels = urllib.request.urlopen('http://localhost:4040/api/tunnels').read().decode('utf8')
spark_ui_url = json.loads(ngrok_tunnels)['tunnels'][0]['public_url']
print("Spark UI:", spark_ui_url)

## Other Imports

In [ ]:
import sys
import urllib.request as req
import zipfile
import glob
from datetime import datetime
from fractions import Fraction
from decimal import Decimal

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

from pyspark import StorageLevel
from pyspark.sql import Row
from pyspark.sql import functions as F
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, StructType, StructField, StringType, IntegerType
from pyspark.sql.window import Window

# Part A - Dataset (5 points)

## Loading the dataset

In [ ]:
url = 'http://files.grouplens.org/datasets/movielens/ml-20m.zip'
filehandle, _ = req.urlretrieve(url)
zip_file_object = zipfile.ZipFile(filehandle, 'r')
zip_file_object.namelist()
zip_file_object.extractall()

In [ ]:
!find ml-20m -type f

In [ ]:
movies_path = "ml-20m/movies.csv"
ratings_path = "ml-20m/ratings.csv"

In [ ]:
ratings_df_csv = spark.read.options(header=True, inferSchema=True).csv(ratings_path)
movies_df_csv = spark.read.options(header=True, inferSchema=True).csv(movies_path)

In [ ]:
ratings_df_csv

In [ ]:
ratings_df_csv

## Preprocessing

> In the following sections, we will work with algorithms that may scale more or less well with the size of the dataset. In this section we will develop several functions that will allow you to sample the dataset. Then in the next part of the notebook, it will be up to you to choose which tools you want to apply to your dataset in order to train your models, knowing that we want to have reasonable processing times (in the order of a minute), while having models that 'work' well.
>
> **Test all the functions that process the dataframe on a toy example, like what is done in first data processing question.**

### Question A1

>Write the *ratings_df_csv* and *movies_df_csv* DataFrame in a compressed parquet format.
>
>Reload them from parquet, to make next computations faster.

### Question A2

> Compute an estimation of the whole *ratings_df* dataset size in memory.
>
>Find the amount of partitions used by *ratings_df*.
>
>We try to have partitions not too big (in order not to crash our executors), and not to small (dealing with too many small partitions can lead to issues, on driver side for example). For this reason, a rule of thumb to have partitions of 128MB is okay. Given this, what do you think of the dataframe? Is the amount of partitions okay? Or should we repartition it? What would be the function to use if we need to repartition the dataframe?

### Question A3

>Create a function named *remove_bad_ratings* that takes a rating dataframe as an argument, and returns a dataframe whose ratings are greater or equals to a rating_threshold, with default value of *3.5*.

### Question A4

>Create a function named *sample_users* that takes a rating dataframe as an argument, and returns a dataframe with only a *ratio* of users (we want to keep all ratings from users that we keep) ; default value of *ratio* parameter is *0.1*. Function should be deterministic.

### Question A5

> Create a function named *remove_exotic_movies*, taking a rating dataframe as argument, and that removes all movies which have less than *nb_min_ratings* ; *nb_min_ratings* parameter has a default value of *1000*.

### Question A6

> Compute the following stats on the dataset:
> - Amount of distinct users
> - Amount of distinct movies
> - Total amount of ratings
> - Let r_u be the amount of ratings made by user u. Study the distribution of r_u over all users (quantiles, histogram...)

### Question A7

> Create a function named *remove_old_movies_in_timelines*, that takes a ratings dataframe as parameter, and only keeps the *nb_max_movies* most recent movies seen by each user ; *nb_max_movies* parameter is defaulted at 100.

# Part B - Association Rules (5 points)

## Naive: Recurring pairs

> This approach is simple and not efficient but gives you a baseline and intuition for the next steps.
>
> Morally, what we want to do is:
> - for each user, regroup all the movies they have liked inside a single row. We will call this the 'user timeline'
> - for each user, generate all pairs of movies across their list of movies.
> - for each pair of movies, count the amount of distinct users with this pair.



### Question B1

> Create a function named *compute_timeline*, that takes a ratings dataframe as parameter, and returns the 'user timeline', a dataframe following this schema:
> - userId : integer
> - movies : list[integer] (list of movieId seen by user)
>
> Test it on a toy example.

### Question B2

> Let's imagine that all of our executors have 4GB of memory. If we consider the 'user timeline' dataset where movie ratings are greater or equal than 3.5, is it okay to store list of movie ids inside rows, as far as memory is concerned?

### Question B3

> Create a function named *compute_pairs*, that takes a user timeline dataframe as parameter, and returns a dataframe of movie pairs (generated across all movies of their timeline) following this schema:
> - userId : integer
> - movieId1 : integer
> - movieId2 : integer
>
> You can rely on an udf to generate list of pair of movies from a list of movies.
> 
> Test it on a toy example.

### Question B4

> Let's imagine that all of our executors have 4GB of memory. 
> 
> If we consider If we consider the 'user timeline where movie ratings are greater or equal than 3.5, what will happen when we generate pairs dataframe for this dataset ?
> 
> You need to consider:
> - amount of bytes retained by lists of pairs
> - amount of partitions we have in user timeline
>
> Also, consider what may happen because of skew. We may have all big user timelines inside same partition.

### Question B5

> Create a function named *compute_pair_frequencies*, that takes a movie pair dataframe as parameter, and returns a dataframe of movie pairs and their user count, following this schema:
> - movieId1 : integer
> - movieId2 : integer
> - count : integer
> 
> Dataframe should be **ordered**, with most frequent pairs first.
>
> Test it on a toy example.

### Question B6

> Quickly test the whole algorithm on *ratings_df* or a subset of it.
> 
> How many shuffles for the whole algorithm ?

## A priori

> You can find a good description of Apriori algorithm here:  
> https://en.wikipedia.org/wiki/Apriori_algorithm
>
> Some other resources:  
> [Apriori — Association Rule Mining In-depth Explanation and Python Implementation](https://towardsdatascience.com/apriori-association-rule-mining-explanation-and-python-implementation-290b42afdfc6)

### Question B7

> Implement your own version of A priori to compute most frequent pairs and quickly test it on *ratings_df* or a subset of it.
> 
> You may want to rely on *F.explode* as an alternative to udf.

### Question B8

> Implement your own version of A priori to compute most frequent triplets.
> 
> A this stage of the 'A priori' section, you are probably doing the same thing multiple times. 
> 
> Maybe it's time to factorize your code...

## 3. FP-Growth

> You can find a good description of FP-Growth algorithm with Spark here:  
> https://spark.apache.org/docs/latest/ml-frequent-pattern-mining.html
>
> Some other resources:  
[FP Growth — Frequent Pattern Generation in Data Mining with Python Implementation](https://towardsdatascience.com/fp-growth-frequent-pattern-generation-in-data-mining-with-python-implementation-244e561ab1c3)

### Question B9

> Use the Spark version of FP-Growth

# Part C - Probabilistic Latent Semantic Model (5 points)

> Aim of this section is to implement a Probabilistic Latent Semantic Model.
> 
> We will use an expectation maximization algorithm to learn its parameters.
>
> In the first set of questions you will implement some utility functions to deal with matrix manipulations.
>
> In the second set of questions, you will implement the algorithm itself.

## Matrix manipulation functions

> We will implement matrix operations that will be usefull to run the PLSI algorithm afterwards.

### Question CMatrix1

#### `matrix_sum_rows` 

> Takes a matrix (a column containing arrays of fixed length) and returns the sum of each row.

In [ ]:
# Hint: https://stackoverflow.com/a/57448698/2015762
def matrix_sum_rows(col_name, length_of_array):
    # ...
    pass

input_array = np.array([[1, 2, 3, 4], [40, 30, 20, 10]], dtype=float)
expected_output = input_array.sum(axis=1)
print('Input array')
print(input_array)
print('Expected output')
print(expected_output)
print('Obtained output')
(
    spark.sparkContext.parallelize(input_array.tolist()).map(lambda x: Row(matrix=x)).toDF()
    .withColumn('row_sum', matrix_sum_rows('matrix', 4))
).show(truncate=False)

### Question CMatrix2

#### `matrix_sum_columns`

> Takes a matrix (a column containing arrays of fixed length) and returns the sum of each column.

In [ ]:
# Hint: https://stackoverflow.com/a/54382990/2015762
def matrix_sum_columns(col_name, length_of_array):
    # ...
    pass

input_array = np.array([[1, 2, 3, 4], [40, 30, 20, 10]], dtype=float)
expected_output = input_array.sum(axis=0)
print('Input array')
print(input_array)
print('Expected output')
print(expected_output)
print('Obtained output')
(
    spark.sparkContext.parallelize(input_array.tolist()).map(lambda x: Row(matrix=x)).toDF()
    .select(matrix_sum_columns('matrix', 4).alias('col_sum'))
).show(truncate=False)

### Question CMatrix3

#### `matrix_normalize_rows`

> Takes a matrix (a column containing arrays of fixed length) and returns the same matrix where the rows have been divded by their sum, such that each row sums to 1.

In [ ]:
def matrix_normalize_rows(col_name, length_of_array):
    # ...
    pass

input_array = np.array([[1, 2, 3, 4], [40, 30, 20, 10]], dtype=float)
expected_output = input_array / input_array.sum(axis=1).reshape(-1, 1)
print('Input array')
print(input_array)
print('Expected output')
print(expected_output)
print('Obtained output')
(
    spark.sparkContext.parallelize(input_array.tolist()).map(lambda x: Row(numbers=x)).toDF()
    .withColumn('normalized_elements', matrix_normalize_rows('numbers', 4))
).show(truncate=False)

### Question CMatrix4

#### `matrix_elementwise_product`

> Takes two matrices and return their elementwise product (aka. Hadamard product)

In [ ]:
def matrix_elementwise_product(col_name_1, col_name_2, length_of_array):
    # ...
    pass

input_array_1 = np.array([[1, 2, 3, 4], [40, 30, 20, 10]], dtype=float)
input_array_2 = np.array([[1, 2, 1, 2], [10, 20, 10, 20]], dtype=float)
expected_output = input_array_1 * input_array_2
print('Input array')
print(input_array_1)
print(input_array_2)
print('Expected output')
print(expected_output)
print('Obtained output')
(
    spark.sparkContext.parallelize(zip(input_array.tolist(), input_array_2.tolist())).map(lambda x: Row(numbers_1=x[0], numbers_2=x[1])).toDF()
    .withColumn('elementwise_products', matrix_elementwise_product('numbers_1', 'numbers_2', 4))
).show(truncate=False)

### Question CMatrix5

#### `matrix_elementwise_divide`

> Takes two matrices and divide elementwise the first one by the second one.

In [ ]:
def matrix_elementwise_divide(col_name_1, col_name_2, length_of_array):
    # ...
    pass

input_array_1 = np.array([[1, 2, 3, 4], [40, 30, 20, 10]], dtype=float)
input_array_2 = np.array([[1, 2, 1, 2], [10, 20, 10, 20]], dtype=float)
expected_output = input_array_1 / input_array_2
print('Input array')
print(input_array_1)
print(input_array_2)
print('Expected output')
print(expected_output)
print('Obtained output')
(
    spark.sparkContext.parallelize(zip(input_array.tolist(), input_array_2.tolist())).map(lambda x: Row(numbers_1=x[0], numbers_2=x[1])).toDF()
    .withColumn('elementwise_divided', matrix_elementwise_divide('numbers_1', 'numbers_2', 4))
).show(truncate=False)

## PLSI

> With
> * N the number of users u
> * M the number of movies s
> * L the number of latent classes z
> * T number of (user, movie) interactions (each interaction (s_t, u_t) means user u_t liked movie s_t)
>
>We suppose that the probability that a user will like a movie can be written in the form of a mixture model given by the equation:
$$
p(s|u) = \sum_{z=1}^L p(s|z) p(z|u)
$$
And we want to optimize the likelihood of the observed user interactions
$$
L = - \frac{1}{T} \sum_{1}^{T} \log p(s_t|u_t) = - \frac{1}{T} \sum_{1}^{T} \sum_{z=1}^L p(s_t|z) p(z|u_t)
$$
That can be done using an EM algorithm working as follow:
>
>**E step**
>
>For each interaction (u_t, s_t), compute for all z = 1, ..., L:
$$
p(z|(u_t, s_t)) = \frac{p(s_t|z) p(z|u_t)}{\sum_z p(s_t|z) p(z|u_t)}
$$
>
>**M step**
>
>Find each movie probability given a latent class
$$
p(s|z) = \frac{N(z, s)}{N(z)} 
\quad \text{where} \quad N(z, s) = \sum_s \sum_u p(z|(u, s)) 
\quad \text{and} \quad N(z) = \sum_s N(z, s)
$$
Find each latent class probability given each user.
$$
p(z|u) = \frac{\sum_s p(z|(u, s))}{\sum_z \sum_s p(z|(u, s))}
$$
>
>We will have the following dataframes
>
>* `count_z_s`: M rows, with columns  `movieId`, `N(z,s)`.
>* `count_z`: 1 row, with column `N(z)`.
>* `p_s_knowing_z`: M rows, with columns  `movieId`, `p(s|z)`. For a given z, the sum of p(s|z) equals 1.
>* `p_z_knowing_u`: N rows, with columns `userId`, `p(z|u)`. For a given u, the sum of p(z|u) equals 1.
>* `p_z_knowing_u_and_s`: N x M rows, with columns `userId`, `movieId`, `p(z|u,s)`.
>
> 
> Implement the PLSI algorithm

### Question CPLSI1.1

In [ ]:
def get_count_z(count_z_s, n_latent_classes):
    """Compute N(z) = sum_s N(z,s)
    """
    # ...
    pass

count_z_s = ss.sparkContext.parallelize([
    st.Row(**{"movieId":1, "N(z,s)": [1., 3., 4.]}),
    st.Row(**{"movieId":2, "N(z,s)": [4., 5., 0.]}),
]).toDF()
get_count_z(count_z_s, 3).show()
# Expected [5., 8., 4.]

### Question CPLSI1.2

In [ ]:
def get_count_z_s(p_z_knowing_u_and_s, n_latent_classes):
    """Compute N(z,s) = sum_u p(z|u,s)
    """
    # ...
    pass

### Question CPLSI1.3

In [ ]:
def get_p_s_knowing_z(count_z_s, count_z, n_latent_classes):
    """Compute p(s|z) = N(z,s) / N(z)
    
    Hint: crossJoin may help
    """
    # ...
    pass

### Question CPLSI1.4

In [ ]:
def get_p_z_knowing_u(p_z_knowing_u_and_s, n_latent_classes):
    """Compute p(z|u) = sum_s p(z|u,s) / sum_z sum_s p(z|u,s)
    """
    # ...
    pass

### Question CPLSI1.5

In [ ]:
def get_p_z_knowing_u_and_s(observed_pairs, count_z_s, count_z, p_z_knowing_u, n_latent_classes):
    """For all pairs of observed (u, s)
    
    Compute p(z|u,s) = [N(z, s) / N(z) * p(z|u)] / sum_z [N(z, s) / N(z) * p(z|u)]
                     = [p(s|z) * p(z|u)] / sum_z [p(s|z) * p(z|u)]
    """
    # ...
    pass

### Question CPLSI1.6

In [ ]:
def log_likelihood(observed_pairs, count_z_s, count_z, p_z_knowing_u, n_latent_classes):
    """Compute the log likelihood of the observed pairs
    
    L = - 1 / T * sum_t log[ p(s|u) ]
      = - 1 / T * sum_t log[ sum_z p(s|z) * p(z|u) ]
    """
   # ...
   pass

### Question CPLSI1.7

In [ ]:
def initialize_statistics(observed_pairs, n_latent_classes):
    """Initialize either p(s|z) and p(z|u) or p(z|(u, s)) to be able to fuel the first iteration of the EM algorithm.
    What would happen if you initialize these to a constant value ?
    """
    # ...
    pass

### Question CPLSI1.8

In [ ]:
def run_plsi(observed_pairs, n_iterations, n_latent_classes, checkpoint_every=1):
    start_init_time = time.time()
    spark.sparkContext.setJobDescription("Initialization")

    # ... = initialize_statistics(observed_pairs, n_latent_classes)
    llh = log_likelihood(observed_pairs, # ... #, n_latent_class
    mlflow.log_metric(key="llh", value=llh, step=0)
    print(f'LLH: {llh:.10f}')
    
    end_init_time = time.time()
    print(f'Initialization: {end_init_time - start_init_time:.1f}s')
    
    for i in range(n_iterations):
        start_e_step = time.time()
        spark.sparkContext.setJobDescription(f"Iteration {i+1}: E-step")
        # E step
        # ...
        
        end_e_step = time.time()
        print(f'Iteration {i+1}: E-step: {end_e_step - start_e_step:.1f}s')
        
        spark.sparkContext.setJobDescription(f"Iteration {i+1}: M-step")
        # M step
        # ...
        
        llh = log_likelihood(observed_pairs, count_z_s, count_z, p_z_knowing_u, n_latent_classes)
        mlflow.log_metric(key="llh", value=llh, step=i+1)
        
        end_m_step = time.time()
        print(f'Iteration {i+1}: M-step: {end_m_step - end_e_step:.1f}s')
        print(f'LLH: {llh:.10f}')
    
    return get_p_s_knowing_z(count_z_s, count_z, n_latent_classes)

In [ ]:
!pip install -q mlflow

In [ ]:
import mlflow
import time

In [ ]:
n_iterations = 20
n_latent_classes = 5

with mlflow.start_run():
    mlflow.log_param("n_iterations", n_iterations)
    mlflow.log_param("n_latent_classes", n_latent_classes)
    run_plsi(ratings_df.sample(0.1), n_iterations=n_iterations, n_latent_classes=n_latent_classes)

### Question CPLSI2.1

> How does the EM algorithm is supposed to scale with the number of EM steps ? Do you observe such a scaling ?


### Question CPLSI2.2
> If each steap takes longer than the previous one: Try using .cache() wisely.

### Question CPLSI2.3

> Try to unpersist your dataframes when they become unneeded (look at the Storage tab in the Spark UI) (Optional + 2pts)

### Question CPLSI2.4

> If after few steps (typically 5), your algorithm starts being much slower and spend more and more time scheduling jobs (look in the Spark UI), try using [.localCheckpoint()](https://spark.apache.org/docs/2.1.0/api/python/pyspark.html#pyspark.RDD.localCheckpoint). How does it differ from caching ? What are the benefits and the drawbacks ?

# Part D - Test them all (5 points)

> In this section, we create training and test datasets, and test all the different prediction algorithms described above.



## Training and Testing datasets

### Question D1

> Create the training dataset named *training_df*. It is made of raw ratings dataframe, where:
> - *hash(userId) % 2 == 0*
> - and *rating >= 3.5*
> 
> You should rely on functions written in Part A.
> 
> Persist it on disk.

### Question D2

> Create a function named *create_test_df*, that creates a test dataset from a ratings dataframe ; it only retain the following records:
> - hash(userId) % 2 == 1
> - rating >= 3.5
> 
> Also, the function returns a dataframe structured like this:
> - userid : the user id
> - movies : list[integer] (all the movies in the user timeline minus the *K* most recent ones)
> - label : list[integer] (all the *K* most recent movies in user timeline)
>
> *K* is parameter whose default value is 5.
> 
> Test the test dataset creation on a toy example.
> 
> Create the real test dataset from the whole movieLense dataset. Name it *test_df*. Persist it on disk.

### Question D3

> Use/adapt each of the algorithms defined in previous sections (naïve, a-priori, FP-growth, PLSI) to predict the 5 next movies that will be seen by the user based on previously seen movies.
> 
> Each algorithm can be 'trained' on *training_df* or a subset of it ; choose and justify.
> 
> For each algorithm, make a quick qualitative analysis, to see how relevant recommended movies are. You should rely on *movies_df* for this question.
>
> Then, compare the algorithms with the *test_df*, with metrics like *recall* and *precision at k* (define some methods that compute recall and precision at k from test dataframe and predictions dataframe parameters)